In [17]:
import numpy as np
from scipy import signal


In [214]:
size1=10
size2=5

epsilon=0.00001
a=np.random.randn(size1,size1).astype(np.float32)
b=np.random.randn(size2,size2).astype(np.float32)

f = open("input1.txt", "w")
for elem in a:
    f.write(" ".join(str(num) for num in elem))
    f.write(" ")
f.close();

f=open("input2.txt","w")
for elem in b:
    f.write(" ".join(str(num) for num in elem))
    f.write(" ")
f.close()

In [223]:
pad=1
f=open("output.txt","r")
line=f.readline()
c1=np.array([np.float32(x) for x in line.split()])
f.close()
a1=np.pad(a, (pad,pad), 'constant', constant_values=(0, 0))
c=np.squeeze(signal.correlate2d(a1,b,mode="valid").reshape(-1,1))

In [224]:
c.shape

(64,)

In [225]:
c1.shape

(64,)

In [226]:
c

array([ -2.9141493 ,   4.2992115 ,  -9.053413  ,  -1.5514579 ,
        -9.487047  ,   4.5419292 ,  -0.07503176,   3.7815676 ,
        -3.655309  ,  10.025689  ,   1.2577807 ,   7.718839  ,
         0.702176  ,  -6.2812805 ,  -1.6984166 ,  -3.5752866 ,
         1.0472558 , -11.569494  ,   8.02701   ,   1.5995841 ,
        13.234629  ,   0.6506822 ,   5.1745906 ,  -1.8631438 ,
         1.1188287 ,  -5.381351  ,   2.011237  ,  -3.6081748 ,
         3.077429  ,  -0.15857637,  -2.443803  ,   2.7728133 ,
         4.120267  ,   8.989772  ,   5.421766  ,   4.1829066 ,
        -8.973361  ,   1.0054643 ,   1.2603048 ,  -8.950005  ,
         6.296142  ,  -3.100207  ,   0.82536554,  11.409778  ,
         0.3335744 ,  -2.0005581 ,  -4.5282087 ,  -4.7130837 ,
         2.367745  ,   5.362504  ,  -2.2239354 ,  -0.03634036,
         2.955905  ,   0.86399597,  -5.6126184 ,   1.4317241 ,
        -7.9675503 ,   4.022829  ,   9.657861  ,  -3.7783937 ,
         3.2073107 ,  -7.1140532 ,  -2.353728  ,   2.72

In [227]:
c

array([ -2.9141493 ,   4.2992115 ,  -9.053413  ,  -1.5514579 ,
        -9.487047  ,   4.5419292 ,  -0.07503176,   3.7815676 ,
        -3.655309  ,  10.025689  ,   1.2577807 ,   7.718839  ,
         0.702176  ,  -6.2812805 ,  -1.6984166 ,  -3.5752866 ,
         1.0472558 , -11.569494  ,   8.02701   ,   1.5995841 ,
        13.234629  ,   0.6506822 ,   5.1745906 ,  -1.8631438 ,
         1.1188287 ,  -5.381351  ,   2.011237  ,  -3.6081748 ,
         3.077429  ,  -0.15857637,  -2.443803  ,   2.7728133 ,
         4.120267  ,   8.989772  ,   5.421766  ,   4.1829066 ,
        -8.973361  ,   1.0054643 ,   1.2603048 ,  -8.950005  ,
         6.296142  ,  -3.100207  ,   0.82536554,  11.409778  ,
         0.3335744 ,  -2.0005581 ,  -4.5282087 ,  -4.7130837 ,
         2.367745  ,   5.362504  ,  -2.2239354 ,  -0.03634036,
         2.955905  ,   0.86399597,  -5.6126184 ,   1.4317241 ,
        -7.9675503 ,   4.022829  ,   9.657861  ,  -3.7783937 ,
         3.2073107 ,  -7.1140532 ,  -2.353728  ,   2.72